# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
filepath = os.getcwd() + '/event_data'

file_path_list = glob.glob(os.path.join(filepath,'*.csv'))
    
print('Number of .csv files in the list: {}'.format(len(file_path_list)))

Number of .csv files in the list: 30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Create appropriate tables in the database to address questions given below.

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### The following three questions will be asked.

### 1. Give the artist, song title and song's length in the music app history that was heard during sessionId = x and itemInSession = y. For example, sessionId = 338, and itemInSession  = 4


### 2. Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = x and sessionid = y. For example, userid = 10 and sessionid = 182
    

### 3. Get every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'

### Query 1:
This table will be called `song_session` because the query wants to get song information from a given session. The primary key will be composed of the `sessionId` (since that will be provided in order to retrieve a song from the session) and the `itemInSession` column to make the key unique (because multiple songs can exist in each session. 

In [8]:
create_table = "CREATE TABLE IF NOT EXISTS song_session "
create_table += """(sessionId int, itemInSession int, artist text, 
                    song_title text, song_length float, 
                    PRIMARY KEY (sessionId, itemInSession))"""
try:
    session.execute(create_table)
except Exception as e:
    print(e)         

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # insert line into table
        query = "INSERT INTO song_session (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)
            break

### Use a test query to make sure that the data was properly inserted into the table.

In [11]:
query = "SELECT artist, song_title, song_length FROM song_session WHERE sessionId=338 AND itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2:
I'll make a table called `user_song_info` because we wish to extract the song info and user name for a given user and session id. The primary key will be composed of a partition key of `userId` and `sessionId` with `itemInSession` used as a clustering column. Combining `userId` and `sessionId` into a partition key promotes evenly distributing the data across nodes. The clustering column will ensure that the data is sorted by `itemInSession` as per the query requirement.  
Again, after data insertion, an example query is used as a test.

In [13]:
create_table = "CREATE TABLE IF NOT EXISTS user_song_info "
create_table += """(userId int, sessionId int, itemInSession int, artist text, 
                    song_title text, first_name text, last_name text,  
                    PRIMARY KEY ((userId, sessionId), itemInSession))"""
try:
    session.execute(create_table)
except Exception as e:
    print(e)   
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO user_song_info (userId, sessionId, itemInSession, artist, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
            print(e)
            print('breaking loop')
            break
            
query = "SELECT artist, song_title, first_name, last_name FROM user_song_info WHERE userId=10 AND sessionId=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3:
To address this query I'll create a table simply called `song`. The song title iself will not be unique so userId will also be added to the primary key. Technically, this primary key may also not be unique because a user can listen to the same song multiple times (either in the same session or in different sessions). However, the query only asks which users listen to the song and is not concerned with frequency.  
Again, a test query is used to ensure proper data insertion.

In [14]:
create_table = "CREATE TABLE IF NOT EXISTS song "
create_table += """(song_title text, userId int, first_name text, last_name text,  
                    PRIMARY KEY (song_title, userId))"""
try:
    session.execute(create_table)
except Exception as e:
    print(e)   
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song (song_title, userId, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        try:
            session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            print(e)
            print('breaking loop')
            break
            
query = "SELECT first_name, last_name FROM song WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.first_name, row.last_name) 

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions
Since this is all for practice, the tables will be removed.

In [15]:
try:
    session.execute('DROP TABLE song_session')
except Exception as e:
    print(e)
    
try:
    session.execute('DROP TABLE user_song_info')
except Exception as e:
    print(e)
    
try:
    session.execute('DROP TABLE song')
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [16]:
session.shutdown()
cluster.shutdown()